# Testscript.ipynb

A Jupyter notebook for testing the `ArBuWe.py` module.
Doubles as a *"tutorial"* of sorts.

In [ ]:
## Import module

import ArBuWe as abw

In [ ]:
## Initialize input parameters

shapefile_path = "data/FI.shp"
raster_path = "data/gfa_res_curr_density_epsg4326.tif"
weather_start = "2010-06-01"
weather_end = "2010-06-02"

In [ ]:
## Try creating a `Shapefile` and inspect the uniform raster

import matplotlib.pyplot as plt
ec = (0,0,0,1) # Edge color for the vector plots (black)
fc = (0,0,0,0) # Face color for the vector plots (transparent)
lw = 0.2 # Line width for the vector plots

shp = abw.Shapefile(shapefile_path)

f = abw.plot_layout(shp, shp.raster, dpi=100, title="Shapefile")

In [ ]:
## Create weights for testing.

#import random

#weights = {'KU005':0.24, 'KU010':0.73, 'KU009':0.03,}
#weights = {lid:random.uniform(0,1) for lid in shp.data.index}
weights = {lid:1 for lid in shp.data.index}

In [ ]:
## Try creating the cutout

cutout = abw.prepare_cutout(shp, weather_start, weather_end)

In [ ]:
## Try preparing the `layout` raster and inspect nonzero data.

raster, layout = abw.prepare_layout(shp, cutout, weights, raster_path, resampling=5)

In [ ]:
## Plot the raster and layout
# This is commented out for the moment, as saving the layouts takes some time.

#fig1 = abw.plot_layout(shp, raster, title="Raster", dpi=300)
#fig2 = abw.plot_layout(shp, layout, title="Layout", dpi=300)

# ArBuWe development stuff

The below cells were used for developing ArBuWe to be able to handle xarray-level initial heating/cooling demand calculations, as well as checking that the `atlite`
tracking options work for irradiation functions as well.

The below stuff is most likely useless, but I'm leaving it in as potential examples.

In [ ]:
# Raw diffuse irradiation

diffi = cutout.irradiation(
    orientation={"slope": 0.0, "azimuth": 0.0},
    irradiation="diffuse",
    tracking="vertical",
    layout=layout, # Remove this line to get xarray level results.
)
diffi.plot()

diffi_vertical = cutout.irradiation(
    orientation={"slope": 90.0, "azimuth": 0.0},
    irradiation="diffuse",
    tracking="vertical",
    layout=layout,
)
diffi_vertical.plot()

## DIFFUSE IRRADIATION IS LOWER FOR VERTICAL SURFACES!

ArchetypeBuildingModel.jl doesn't account for this properly, causing more solar gains than there should be.

In [ ]:
# Raw ground irradiation?

ground = cutout.irradiation(
    orientation={"slope": 0.0, "azimuth": 0.0},
    irradiation="ground",
    tracking="vertical",
    layout=layout, # Remove this line to get xarray level results.
)
ground.plot()

ground_vertical = cutout.irradiation(
    orientation={"slope": 90.0, "azimuth": 0.0}, # Ground irradiation is not affected by azimuth
    irradiation="ground",
    tracking="vertical",
    layout=layout,
)
ground_vertical.plot()

In [ ]:
# Direct irradiation?

dirs = {
    "horizontal": (0.0, 0.0),
    "north": (90.0, 0.0),
    "east": (90.0, 90.0),
    "south": (90.0, 180.0),
    "west": (90.0, 270.0),
}

diri_none = {
    dir: cutout.irradiation(
        orientation={"slope": sl, "azimuth": az},
        irradiation="direct",
        layout=layout, # Remove this line to get xarray level results.
    )
    for dir, (sl, az) in dirs.items()
}

diri_vertical = {
    dir: cutout.irradiation(
        orientation={"slope": sl, "azimuth": az},
        irradiation="direct",
        tracking="vertical",
        layout=layout, # Remove this line to get xarray level results.
    )
    for dir, (sl, az) in dirs.items()
}

# Plot comparison, "horizontal" shouldn't depend on tracking.

diri_none["horizontal"].plot()
diri_vertical["horizontal"].plot()

In [ ]:
# Plot comparison, no tracking should show 4 different cardinal directions (plus horizontal).

for dir, (sl, az) in dirs.items():
    diri_none[dir].plot()

In [ ]:
# Plot comparison, vertical axis tracking should result in identical radiation for all cardinal directions (plus horizontal).

for dir, (sl, az) in dirs.items():
    diri_vertical[dir].plot()

In [ ]:
# Plot comparison, vertical axis tracking vs no tracking?

for dir, (sl, az) in dirs.items():
    diri_none[dir].plot()
    diri_vertical[dir].plot()

## Seems reasonable enough.

Vertical axis tracking seems to work as intended in `atlite`.
Since we likely won't ever know the actual distribution of window and envelope surface
areas towards the cardinal directions *(or any direction to be frank)* on the
building-stock-scale, we might as well assume the vertical envelope areas to be more or less equally distributed.

Thus, we should be able to use the `vertical` axis tracking option in `atlite` to
calculate the effective solar irradiation on the vertical envelope.
Naturally, this must take into account that not all of the envelope surface faces
the correct way, but that shouldn't be an issue.


### Next steps: Draft the calculations for the heating/cooling demand

What coefficients do we actually need from ArBuMo, and how to efficiently manage
the weather stuff for thermal mass nodes.


#### Test preprocessing of weather

First, we want to preprocess some weather data.
Mainly, we're interested in the ambient air temperature as well as the effective total
irradiation on horizontal and vertical surfaces.

In [ ]:
# Test weather data preprocessing.

external_shading_coefficient = 0.7

ambient_temperature, effective_irradiation = abw.preprocess_weather(
    cutout, external_shading_coefficient
)

In [ ]:
ambient_temperature[1].plot()

In [ ]:
effective_irradiation['horizontal'][1].plot()

In [ ]:
effective_irradiation['vertical'][1].plot()

#### Generate test internal heat gains

We need these for testing the initial heating demand calculations.

In [ ]:
# Define internal heat gains

import numpy as np

int_gains_W_m2 = np.array([2.17, 2.17, 2.17, 3.47, 5.25, 4.86, 3.20, 1.34, 0.79, 0.79, 0.79, 1.25, 3.29, 3.44, 3.90, 9.78, 8.60, 4.76, 12.75, 7.14, 4.21, 3.55, 2.17, 2.17])
gross_floor_area_m2 = 135.56
int_gains_W = int_gains_W_m2 * gross_floor_area_m2
int_gains_W

In [ ]:
## Expand the array to an xarray

# Format to full-length timeseries
time = ambient_temperature.time
int_array = np.tile(int_gains_W, len(time)//len(int_gains_W))

# Call the expansion function
internal_heat_gains_W = abw.expand_to_xarray(
    int_array,
    ambient_temperature,
    "internal heat gains",
    "W"
)
internal_heat_gains_W

#### Test preprocessing of heating demand.

So the next step is to actually calculate the initial heating demand on `xarray` level.
Fortunately, with this type of an approach, the cooling demand is just the negative part of heating demand, so the same function can be used for both applications.

NOTE! Internal heat gains will need some additional processing to fit the `xarray` format.

In [ ]:
# Define remaining required input data for the heating demand processing.

self_discharge_coefficient_W_K = 0.0
total_ambient_heat_transfer_coefficient_W_K = 118.453
solar_heat_gain_convective_fraction = 0.6
window_non_perpendicularity_correction_factor = 0.9
total_normal_solar_energy_transmittance = 0.472
vertical_window_surface_area_m2 = 25.295
horizontal_window_surface_area_m2 = 0.0

In [ ]:
# Define time-dependent set-point via xarray expansion.

set_points = np.tile([284.15, 294.15], len(time)//2)
set_point_K = abw.expand_to_xarray(
    set_points,
    ambient_temperature,
    "Heating set point",
    "K"
)
set_point_K

In [ ]:
# Test processing initial heating demand.

initial_demand = abw.process_initial_heating_demand(
    set_point_K,
    ambient_temperature,
    effective_irradiation,
    internal_heat_gains_W,
    self_discharge_coefficient_W_K,
    total_ambient_heat_transfer_coefficient_W_K,
    solar_heat_gain_convective_fraction,
    window_non_perpendicularity_correction_factor,
    total_normal_solar_energy_transmittance,
    vertical_window_surface_area_m2,
    horizontal_window_surface_area_m2,
)
initial_demand[1].plot()

#### Test separation and aggregation of heating/cooling demand.

Ok, seems like the heating/cooling demand calculations are working. Next, we'll have to see if we can actually separate aggregate the initial heating and cooling demands.

In [ ]:
## Separate initial demand into heating and cooling.

import xarray as xr

heating_demand = xr.where(initial_demand < 0.0, 0.0, initial_demand)
cooling_demand = xr.where(initial_demand > 0.0, 0.0, -initial_demand)

In [ ]:
# Inspect initial demand

initial_demand.plot()

In [ ]:
# Inspect heating demand

heating_demand.plot()

In [ ]:
cooling_demand.plot()

In [ ]:
initial_demand[2].plot()

In [ ]:
heating_demand[2].plot()

In [ ]:
cooling_demand[2].plot()

In [ ]:
## Try aggregating the weather?

agg_ambient_temperature = abw.aggregate_xarray(ambient_temperature, layout)
comp_temperature = cutout.temperature(layout=layout) + 273.15
agg_ambient_temperature.plot(label="ArBuWe")
comp_temperature.plot(label="atlite")
plt.legend(loc="upper left")

In [ ]:
## Try aggregating the heating and cooling demands.

agg_initial_demand = abw.aggregate_xarray(initial_demand, layout)
agg_heating_demand = abw.aggregate_xarray(heating_demand, layout)
agg_cooling_demand = abw.aggregate_xarray(cooling_demand, layout)

In [ ]:
agg_initial_demand.plot(label="net")
agg_heating_demand.plot(label="heating")
agg_cooling_demand.plot(label="cooling")
plt.legend(loc="upper left")

Surprisingly, the aggregated heating/cooling demand calculations seem to be working as intended. The zig-zag is a consequence of time-varying heating/cooling set points, which one would never do in practise. This is purely for testing purposes.

Furthermore, the aggregation of ambient temperature seems to result in identical values regardless of whether I use my own aggregation or PyPSA/atlite native aggregation functionality! *(Then again, it's basically just copy-paste from atlite, but at least I didn't mess anything up!)*

#### Test the master processing function

Finally, we'll need to test the master processing function aimed at the interface with ArBuMo.jl. Essentially, we'll need to return the initial heating and cooling demands, as well as the aggregated weather quantities for structural thermal mass dynamics in ArBuMo.jl.

In [ ]:
## Test the master processing function

hd, cd, aT, irr = abw.aggregate_demand_and_weather(
    shapefile_path,
    weather_start,
    weather_end,
    weights,
    external_shading_coefficient,
    set_points,
    set_points + 5,
    int_array,
    int_array - 5,
    np.tile([self_discharge_coefficient_W_K], len(time)),
    np.tile([total_ambient_heat_transfer_coefficient_W_K], len(time)),
    np.tile([total_ambient_heat_transfer_coefficient_W_K * 2.0], len(time)),
    solar_heat_gain_convective_fraction,
    window_non_perpendicularity_correction_factor,
    total_normal_solar_energy_transmittance,
    vertical_window_surface_area_m2,
    horizontal_window_surface_area_m2,
    raster_path=None,
    resampling=5,
    filename="scope",
    save_layouts=True,
)

In [ ]:
hd.plot(label="heating")
cd.plot(label="cooling")
plt.legend(loc="upper left")

In [ ]:
aT.plot()

In [ ]:
irr["horizontal"].plot(label="horizontal")
irr["vertical"].plot(label="vertical")
plt.legend(loc="upper left")

The horizontal irradiation appears much higher here compared to the vertical irradiation because the effective vertical irradiation is to be applied to all vertical surface area, not just the part that is actually facing the sun. For vertical surfaces matching the solar azimuth, the irradiation is Pi times higher.

## Conclusion: it works?

I almost can't believe it, but it seems that my initial heating and cooling demand processing is working as expected. I'm not sure if it will be fast enough to do what is required of it on the full Finnish/European scales, but you never know.
There are actually things wrong with the old ArchetypeBuildingModel:
 - Diffuse irradiation is not identical for horizontal and vertical surfaces, which overestimates solar gains in ArchetypeBuildingModel.
 - Ground irradiation is neglegted in ArchetypeBuildingModel, which compensates for the above slightly, but not enough.